In [1]:
# Required: Import packages
import json
import pandas as pd
from PIL import Image
import numpy as np


In [2]:
!pwd

/Users/tiffanyshih/Documents/Coursework/MIDS_207/w207_final_project


# 0. Load and Clean Data

In [3]:
# REQUIRED: Load labels json file containing image object labels

with open('../Final_Project/bdd100k_drivable_labels_trainval/labels/bdd100k_labels_images_train.json') as json_file:
    train_data = json.load(json_file)

with open('../Final_Project/bdd100k_drivable_labels_trainval/labels/bdd100k_labels_images_val.json') as json_file:
    test_data = json.load(json_file)

In [4]:
# REQUIRED: Normalize semi-structured JSON data into a pd dataframe
data_normalized = pd.json_normalize(train_data)
test_data_normalized = pd.json_normalize(test_data)
type(data_normalized)

pandas.core.frame.DataFrame

In [7]:
# Optional - Testing only -- check number of images 
data_normalized.shape

(69863, 6)

In [13]:
# Optional - Testing only -- check format and contents of df
data_normalized.head()

,name,timestamp,labels,attributes.weather,attributes.scene,attributes.timeofday
0,0000f77c-6257be58.jpg,10000,"[{'category': 'traffic light', 'attributes': {...",clear,city street,daytime
1,0000f77c-62c2a288.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,dawn/dusk
2,0000f77c-cb820c98.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,residential,dawn/dusk
3,0001542f-5ce3cf52.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,city street,night
4,0001542f-7c670be8.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,night


In [14]:
# Optional - Testing only -- check the df values corresponding to a specific image file name
data_normalized[data_normalized['name'] == '0a0a0b1a-7c39d841.jpg']


,name,timestamp,labels,attributes.weather,attributes.scene,attributes.timeofday
3915,0a0a0b1a-7c39d841.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime


In [ ]:
# Optional - Testing only -- check what format the "labels" contains
type(data_normalized[data_normalized['name'] == '0a0a0b1a-7c39d841.jpg'].labels)
data_normalized.iloc[3915].labels

In [8]:
# Optional - Check size of test and validation data for conditions clear, highway, and daytime.
train_shape = data_normalized[(data_normalized['attributes.weather'] == "clear") & 
               (data_normalized['attributes.scene'] == "highway") & 
               (data_normalized['attributes.timeofday'] == "daytime")].shape

test_shape = test_data_normalized[(test_data_normalized['attributes.weather'] == "clear") & 
               (test_data_normalized['attributes.scene'] == "highway") & 
               (test_data_normalized['attributes.timeofday'] == "daytime")].shape

In [9]:
# Optional - Print out number of images in train and test datasets

print("Train Data: We get a total of %d images taken during clear weather, highway, and daytime." %train_shape[0])
print("Test Data: We get a total of %d images taken during clear weather, highway, and daytime." %test_shape[0])
print("Test data composes of %.2f%% of total dataset" %(100*test_shape[0]/(test_shape[0]+train_shape[0])))

Train Data: We get a total of 3575 images taken during clear weather, highway, and daytime.
Test Data: We get a total of 514 images taken during clear weather, highway, and daytime.
Test data composes of 12.57% of total dataset


# 1. Determine number of objects in the image

In [5]:
# REQUIRED: Function used to clean data, extract labels, and extract data for sunny, 
# highway, and daytime conditions

def clean_data(data_normalized):
    # Initialize lists of object types. Each list will contain 69863 values, each of which corresponds to an image specified as a row in data_cleaned 
    num_objects = []
    num_road = []
    num_sidewalk = []
    num_building = []
    num_wall = []
    num_fence = []
    num_pole = []
    num_traffic_light = []
    num_traffic_sign = []
    num_vegetation = []
    num_terrain = []
    num_sky = []
    num_person = []
    num_rider = []
    num_car = []
    num_truck = []
    num_bus = []
    num_train = []
    num_motorcycle = []
    num_bicycle = []
    num_lanes = []
    num_drivable_area = []

    # Iterate through each image in data_normalized, sum up the number of objects corresponding to each object type, and append to
    # the corresponding object list
    for index, row in data_normalized.iterrows():
        num_objects.append(len(row['labels']))
        num_road.append(sum(x['category'] == 'road' for x in row['labels']))
        num_sidewalk.append(sum(x['category'] == 'sidewalk' for x in row['labels']))
        num_building.append(sum(x['category'] == 'building' for x in row['labels']))
        num_wall.append(sum(x['category'] == 'wall' for x in row['labels']))
        num_fence.append(sum(x['category'] == 'fence' for x in row['labels']))
        num_pole.append(sum(x['category'] == 'pole' for x in row['labels']))
        num_traffic_light.append(sum(x['category'] == 'traffic light' for x in row['labels']))
        num_traffic_sign.append(sum(x['category'] == 'traffic sign' for x in row['labels']))
        num_vegetation.append(sum(x['category'] == 'vegetation' for x in row['labels']))
        num_terrain.append(sum(x['category'] == 'terrain' for x in row['labels']))
        num_sky.append(sum(x['category'] == 'sky' for x in row['labels']))
        num_person.append(sum(x['category'] == 'person' for x in row['labels']))
        num_rider.append(sum(x['category'] == 'rider' for x in row['labels']))
        num_car.append(sum(x['category'] == 'car' for x in row['labels']))
        num_truck.append(sum(x['category'] == 'truck' for x in row['labels']))
        num_bus.append(sum(x['category'] == 'bus' for x in row['labels']))
        num_train.append(sum(x['category'] == 'train' for x in row['labels']))
        num_motorcycle.append(sum(x['category'] == 'motorcycle' for x in row['labels']))
        num_bicycle.append(sum(x['category'] == 'bicycle' for x in row['labels']))
        num_lanes.append(sum(x['category'] == 'lane' for x in row['labels']))
        num_drivable_area.append(sum(x['category'] == 'drivable area' for x in row['labels']))

    data_normalized['num_objects'] = num_objects
    data_normalized['num_sidewalk'] = num_sidewalk
    data_normalized['num_building'] = num_building
    data_normalized['num_wall'] = num_wall
    data_normalized['num_fence'] = num_fence
    data_normalized['num_pole'] = num_pole
    data_normalized['num_traffic_light'] = num_traffic_light
    data_normalized['num_traffic_sign'] = num_traffic_sign
    data_normalized['num_vegetation'] = num_vegetation
    data_normalized['num_terrain'] = num_terrain
    data_normalized['num_sky'] = num_sky
    data_normalized['num_person'] = num_person
    data_normalized['num_rider'] = num_rider
    data_normalized['num_car'] = num_car
    data_normalized['num_truck'] = num_truck
    data_normalized['num_bus'] = num_bus
    data_normalized['num_train'] = num_train
    data_normalized['num_motorcycle'] = num_motorcycle
    data_normalized['num_bicycle'] = num_bicycle
    data_normalized['num_lanes'] = num_lanes
    data_normalized['num_drivable_area'] = num_drivable_area

    # Filter only for images where weather == "clear", scene == "highway", and timeofday == "daytime". 
    # Note: A better implementation may be to filter the dataframe prior to counting the objects per image, 
    # but it may be useful to keep the data_normalized and object counts for each image in case we want to use different filtering values.

    data_cleaned = data_normalized[(data_normalized['attributes.weather'] == "clear") & 
                   (data_normalized['attributes.scene'] == "highway") & 
                   (data_normalized['attributes.timeofday'] == "daytime")]
    return data_cleaned

In [6]:
# Clean Training & Validation Data
training_data_cleaned = clean_data(data_normalized)
training_data_cleaned

,name,timestamp,labels,attributes.weather,attributes.scene,attributes.timeofday,num_objects,num_sidewalk,num_building,num_wall,...,num_person,num_rider,num_car,num_truck,num_bus,num_train,num_motorcycle,num_bicycle,num_lanes,num_drivable_area
10,00067cfb-caba8a02.jpg,10000,"[{'category': 'traffic light', 'attributes': {...",clear,highway,daytime,29,0,0,0,...,0,0,3,0,1,0,0,0,11,3
62,002d290d-89f4e5c0.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,23,0,0,0,...,0,0,6,0,0,1,0,0,10,2
88,003baca5-aab2e274.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,6,0,0,0,...,0,0,2,0,0,0,0,0,1,1
89,003baca5-ad660439.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,15,0,0,0,...,0,0,10,1,0,0,0,0,2,1
92,003e23ee-07d32feb.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,28,0,0,0,...,0,0,6,1,1,0,0,0,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69675,b18cb922-e3af77af.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,18,0,0,0,...,0,0,5,0,0,0,0,0,9,2
69805,fde2db45-f6e2fbd1.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,11,0,0,0,...,0,0,4,0,0,0,0,0,6,1
69809,fde816b0-1b0f1a85.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,29,0,0,0,...,0,0,8,0,0,0,0,0,9,2
69812,fdebe7ab-8409a734.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,40,0,0,0,...,0,0,12,0,0,0,0,0,8,2


In [7]:
# Clean Test Data
test_data_cleaned = clean_data(test_data_normalized)
test_data_cleaned

,name,timestamp,labels,attributes.weather,attributes.scene,attributes.timeofday,num_objects,num_sidewalk,num_building,num_wall,...,num_person,num_rider,num_car,num_truck,num_bus,num_train,num_motorcycle,num_bicycle,num_lanes,num_drivable_area
13,b1cebfb7-284f5117.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,21,0,0,0,...,0,0,11,3,0,0,0,0,6,1
40,b1d7b3ac-5af8623b.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,20,0,0,0,...,0,0,8,0,1,0,0,0,2,1
55,b1df722f-57d21f3f.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,25,0,0,0,...,0,0,8,3,0,0,0,0,9,4
56,b1df722f-5bcc3db7.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,38,0,0,0,...,0,4,12,1,0,0,0,0,9,5
67,b1e8ad72-c3c79240.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,30,0,0,0,...,0,0,7,0,0,0,0,0,13,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9787,c9e7c9d4-18d31194.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,34,0,0,0,...,0,0,19,2,0,0,0,0,9,4
9838,ca077fe9-d2cf4446.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,19,0,0,0,...,0,0,3,0,0,0,0,0,6,2
9858,ca146661-7f70d0c4.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,37,0,0,0,...,0,0,18,0,0,0,0,0,9,3
9897,ca2bcf0e-69e964d2.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,29,0,0,0,...,0,0,13,0,0,0,0,0,7,3


In [8]:
# Split training dataset to training and validation

training_data_split = training_data_cleaned.sample(frac = 0.9)
val_data_split = training_data_cleaned.drop(training_data_split.index)
val_data_split

,name,timestamp,labels,attributes.weather,attributes.scene,attributes.timeofday,num_objects,num_sidewalk,num_building,num_wall,...,num_person,num_rider,num_car,num_truck,num_bus,num_train,num_motorcycle,num_bicycle,num_lanes,num_drivable_area
95,004071a4-4e8a363a.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,29,0,0,0,...,0,0,11,0,0,0,0,0,8,4
159,006fdb67-f4820206.jpg,10000,"[{'category': 'bus', 'attributes': {'occluded'...",clear,highway,daytime,40,0,0,0,...,5,0,9,0,1,0,0,0,6,2
275,00b04b30-501822fa.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,46,0,0,0,...,0,0,25,0,0,0,0,0,10,6
417,011f8481-66729fb4.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,19,0,0,0,...,0,0,10,0,0,0,0,0,6,2
469,013aba02-3fbd03ff.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,30,0,0,0,...,0,0,11,2,6,0,0,0,6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69089,b0253ab0-7f908976.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,25,0,0,0,...,0,0,7,1,0,0,0,0,10,4
69134,b03d18af-a23f2e42.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,24,0,0,0,...,0,0,10,1,0,0,0,0,5,2
69163,b0521762-565e20cd.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,33,0,0,0,...,0,0,16,0,0,0,0,0,8,3
69465,b10af931-d3a3f406.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,29,0,0,0,...,0,0,14,0,0,0,0,0,9,5


In [15]:
training_data_split

,name,timestamp,labels,attributes.weather,attributes.scene,attributes.timeofday,num_objects,num_sidewalk,num_building,num_wall,...,num_person,num_rider,num_car,num_truck,num_bus,num_train,num_motorcycle,num_bicycle,num_lanes,num_drivable_area
26905,44ab2984-262d20dd.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,15,0,0,0,...,0,0,3,0,1,0,0,0,6,3
54269,8ac0c658-872dcdfd.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,19,0,0,0,...,0,0,4,0,0,0,0,0,11,3
59980,999d99fa-6bd7c5a8.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,28,0,0,0,...,0,0,18,0,0,0,0,0,6,1
19052,30853278-13d158f8.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,55,0,0,0,...,0,0,44,0,0,0,0,0,4,1
20688,34cbfd92-77b10b17.jpg,10000,"[{'category': 'traffic sign', 'attributes': {'...",clear,highway,daytime,11,0,0,0,...,0,0,4,1,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,003baca5-ad660439.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,15,0,0,0,...,0,0,10,1,0,0,0,0,2,1
8802,16b0973f-b62dae5e.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,23,0,0,0,...,0,0,12,2,0,0,0,0,8,1
43017,6e1ecbc2-efe66c61.jpg,10000,"[{'category': 'traffic light', 'attributes': {...",clear,highway,daytime,28,0,0,0,...,0,0,7,1,0,0,0,0,9,2
52974,87a9cad2-a1655d97.jpg,10000,"[{'category': 'car', 'attributes': {'occluded'...",clear,highway,daytime,17,0,0,0,...,0,0,13,0,0,0,0,0,3,1


In [ ]:
# REQUIRED - ONE TIME ONLY - ONLY PERFORM THIS STEP ONCE
# Move training and validation files to a separate folder. 

import os
import shutil

source = '../Final_Project/bdd100k_images_100k/images/100k/train/'
destination = '../Final_Project/bdd100k_images_100k/images/100k/train_clean/'

for f in training_data_split['name']:
    origin = source + f
    dest = destination + f
    shutil.move(origin, dest)


In [12]:
# REQUIRED - ONE TIME ONLY - ONLY PERFORM THIS STEP ONCE
# Move cleaned validation dataset to new folder

source = '../Final_Project/bdd100k_images_100k/images/100k/train/'
destination = '../Final_Project/bdd100k_images_100k/images/100k/val_clean/'

for f in val_data_split['name']:
    origin = source + f
    dest = destination + f
    shutil.move(origin, dest)

In [14]:
# REQUIRED - ONE TIME ONLY - ONLY PERFORM THIS STEP ONCE
# Move cleaned test dataset to new folder

source = '../Final_Project/bdd100k_images_100k/images/100k/val/'
destination = '../Final_Project/bdd100k_images_100k/images/100k/test_clean/'

for f in test_data_cleaned['name']:
    origin = source + f
    dest = destination + f
    shutil.move(origin, dest)
    

In [26]:
# REQUIRED - ONE TIME ONLY - ONLY PERFORM THIS STEP ONCE
# Move masks training files to a separate folder. 

source = '../Final_Project/bdd100k_sem_masks_labels_release/labels/drivable/masks/train/'
destination = '../Final_Project/bdd100k_sem_masks_labels_release/labels/drivable/masks/train_clean/'

for f in training_data_split['name']:
    origin = source + f[:-3] + 'png'
    dest = destination + f[:-3] + 'png'
    shutil.move(origin, dest)

In [29]:
# REQUIRED - ONE TIME ONLY - ONLY PERFORM THIS STEP ONCE
# Move masks validation files to a separate folder. 

source = '../Final_Project/bdd100k_sem_masks_labels_release/labels/drivable/masks/train/'
destination = '../Final_Project/bdd100k_sem_masks_labels_release/labels/drivable/masks/val_clean/'

for f in val_data_split['name']:
    origin = source + f[:-3] + 'png'
    dest = destination + f[:-3] + 'png'
    shutil.move(origin, dest)

In [30]:
# REQUIRED - ONE TIME ONLY - ONLY PERFORM THIS STEP ONCE
# Move masks test files to a separate folder. 

source = '../Final_Project/bdd100k_sem_masks_labels_release/labels/drivable/masks/val/'
destination = '../Final_Project/bdd100k_sem_masks_labels_release/labels/drivable/masks/test_clean/'

for f in test_data_cleaned['name']:
    origin = source + f[:-3] + 'png'
    dest = destination + f[:-3] + 'png'
    shutil.move(origin, dest)

# 2. Set up CNN Model

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam

In [18]:
from warnings import filterwarnings
filterwarnings('ignore')

In [19]:
classifier = Sequential()
classifier.add(Conv2D(32,(3,3),input_shape=(720,1280,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2),strides=2)) #if stride not given it equal to pool filter size
classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2),strides=2))
classifier.add(Flatten())
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=1,activation='sigmoid'))
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
classifier.compile(optimizer=adam,loss='binary_crossentropy',metrics=['accuracy'])
#tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

In [20]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 718, 1278, 32)     896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 359, 639, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 357, 637, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 178, 318, 32)     0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1811328)           0         
                                                                 
 dense (Dense)               (None, 128)               2

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

seed = 1
#Training Set
train_set = train_datagen.flow_from_directory('../Final_Project/bdd100k_sem_masks_labels_release/labels/drivable/masks/train_clean/',
                                             target_size=(720,1280),
                                             batch_size=1,
                                             class_mode='categorical')
mask_generator = mask_datagen.flow_from_directory("dataset/mask/",
                                                   class_mode=None, seed=seed)

#Validation Set
val_set = test_datagen.flow_from_directory('../Final_Project/bdd100k_sem_masks_labels_release/labels/drivable/masks/val_clean/',
                                           target_size=(720,1280),
                                           batch_size = 1,
                                           class_mode='categorical',
                                           shuffle=False)
#Test Set /no output available
test_set = test_datagen.flow_from_directory('../Final_Project/bdd100k_sem_masks_labels_release/labels/drivable/masks/test_clean/',
                                            target_size=(720,1280),
                                            batch_size=1,
                                            shuffle=False)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


# 3. Determine the pixel locations of the objects in the image, specifically drivable area

/Users/tiffanyshih/Documents/Coursework/MIDS_207/w207_final_project


# 4. Opening an image file and converting contents to raw pixel values

In [4]:
# Optional: Open sample image and convert to raw pixel values

# with open('../Final_Project/bdd100k/labels/bdd100k_labels_images_train.json') as json_file:

# load the image
with Image.open('../Final_Project/images/10k/train/0a0a0b1a-7c39d841.jpg') as image:
    # convert image to numpy array
    image_data = np.asarray(image)
    
# Print image_data type (np array)
print(type(image_data))

# summarize shape/dimensions (should be height, width, and 
print(image_data.shape)

# Print image data
print(image_data) 

# Convert image data back to a Pillow image
image2 = Image.fromarray(image_data)
print(type(image2))

# summarize image details
print(image2.mode)
print(image2.size)

# Open PIL image 
image2.show()

<class 'numpy.ndarray'>
(720, 1280, 3)
[[[ 55 133 199]
  [ 55 133 199]
  [ 55 133 199]
  ...
  [ 96 178 238]
  [ 95 177 237]
  [ 95 177 237]]

 [[ 55 133 199]
  [ 55 133 199]
  [ 55 133 199]
  ...
  [ 96 178 238]
  [ 95 177 237]
  [ 95 177 237]]

 [[ 56 134 200]
  [ 56 134 200]
  [ 56 134 200]
  ...
  [ 96 178 238]
  [ 95 177 237]
  [ 95 177 237]]

 ...

 [[ 10  34  46]
  [ 10  34  46]
  [ 10  34  46]
  ...
  [  3  14  18]
  [  3  14  18]
  [  3  14  18]]

 [[  7  31  43]
  [  7  31  43]
  [  7  31  43]
  ...
  [  4  15  19]
  [  4  15  19]
  [  4  15  19]]

 [[  8  32  44]
  [  8  32  44]
  [  8  32  44]
  ...
  [  4  15  19]
  [  4  15  19]
  [  4  15  19]]]
<class 'PIL.Image.Image'>
RGB
(1280, 720)


In [37]:
# Optional: Open bitmask image and convert to raw pixel values

# with open('../Final_Project/bdd100k/labels/bdd100k_labels_images_train.json') as json_file:

# load the image
with Image.open('../Final_Project/bdd100k_sem_masks_labels_release/labels/drivable/masks/train_clean/0a0a0b1a-7c39d841.png') as image:
    # convert image to numpy array
    image_data = np.asarray(image)
    
# Print image_data type (np array)
print(type(image_data))

# summarize shape/dimensions (should be height, width, and 
print(image_data.shape)

# Print image data
print(image_data) 

print("mean image data:",np.mean(image_data))
print("min image data:",np.min(image_data))
print("max image data:",np.max(image_data))
print("Unique image data:", np.unique(image_data))
# # Convert image data back to a Pillow image
# image2 = Image.fromarray(image_data)
# print(type(image2))

# # summarize image details
# print(image2.mode)
# print(image2.size)

# # Open PIL image 
# image2.show()

<class 'numpy.ndarray'>
(720, 1280)
[[2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 ...
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]
 [2 2 2 ... 2 2 2]]
mean image data: 1.6110405815972222
min image data: 0
max image data: 2
Unique image data: [0 1 2]


In [ ]:
for filename in data_cleaned['name']:
    # load the image
    filepath = '../Final_Project/images/10k/train/'
    with Image.open(filename + filepath) as image:
        # convert image to numpy array
        image_data = np.asarray(image)

'../Final_Project/images/10k/train/0a0a0b1a-7c39d841.jpg'